In [1]:
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

## Defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'data-448'
bucket = s3.Bucket(bucket_name)

## Defining the file to be read from s3 bucket
file_key = 'Chapter6/Credit_Card.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## Reading the csv file
credit = pd.read_csv(file_content_stream)
credit = credit.drop(columns = ['ID'], axis = 1)
credit.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,20000.0,2,2,1,24,2,2,-1,-1,-2,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,120000.0,2,2,2,26,-1,2,0,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,90000.0,2,2,2,34,0,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,50000.0,2,2,1,37,0,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,50000.0,1,2,1,57,-1,0,-1,0,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [19]:
credit.columns

Index(['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default.payment.next.month'],
      dtype='object')

In [20]:
credit.shape

(30000, 24)

In [21]:
credit['default.payment.next.month'].value_counts() / credit.shape[0]

0    0.7788
1    0.2212
Name: default.payment.next.month, dtype: float64

## Splitting the data

In [22]:
## Defining the input and target variables
X = credit.drop(columns = ['default.payment.next.month'], axis = 1)
Y = credit['default.payment.next.month']

## Splitting the data into train and validation
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.2, stratify = Y)

## Splitting into validation and test
X_val, X_test, Y_val, Y_test = train_test_split(X_val, Y_val, test_size = 0.5, stratify = Y_val)

## Standardizing the data

In [23]:
## Loading minmax
scaler = MinMaxScaler()

## Transforming to 0-1 the inputs 
X_train = scaler.fit_transform(X_train)
X_val = scaler.fit_transform(X_val)
X_test = scaler.fit_transform(X_test)

## Buiding the base learners

In [29]:
## Decision tree
tree = DecisionTreeClassifier(max_depth = 3).fit(X_train, Y_train)
pred_tree = tree.predict_proba(X_val)[:, 1]

## Nearest-Neighbors
knn = KNeighborsClassifier(n_neighbors = 5).fit(X_train, Y_train)
pred_knn = knn.predict_proba(X_val)[:, 1]

## support vector machine 
svm = SVC(kernel = 'rbf', probability = True).fit(X_train, Y_train)
pred_svm = svm.predict_proba(X_val)[:, 1]

## Creating the new inputs and target

In [30]:
## Creating a data-frame with the base learners predictions
X_preds = pd.DataFrame({'tree': pred_tree, 'knn': pred_knn, 'svm': pred_svm})
X_preds.head()

,tree,knn,svm
0,0.114510,0.0,0.160942
1,0.114510,0.0,0.181479
2,0.114510,0.2,0.171468
3,0.215915,0.2,0.178212
4,0.215915,0.4,0.210031


## Building the meta-learner

In [31]:
## building the meta-learner
meta_learner = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_preds, Y_val)

## Predicting on the test datasets with base learners

In [33]:
## Predicting on the test datasets
test_pred_tree = tree.predict_proba(X_test)[:, 1]
test_pred_knn = knn.predict_proba(X_test)[:, 1]
test_pred_svm = svm.predict_proba(X_test)[:, 1]

## Creating a data-frame with base learners 
X_test_preds = pd.DataFrame({'tree': test_pred_tree, 'knn': test_pred_knn, 'svm': test_pred_svm})
X_test_preds.head()

,tree,knn,svm
0,0.11451,0.0,0.173227
1,0.11451,0.2,0.191382
2,0.11451,0.0,0.171401
3,0.11451,0.2,0.138620
4,0.11451,0.2,0.186879


In [36]:
## Changing likelihoods to labels
test_pred_tree_label = np.where(test_pred_tree < 0.25, 0, 1) 
test_pred_knn_label = np.where(test_pred_knn < 0.25, 0, 1)
test_pred_svm_label = np.where(test_pred_svm < 0.25, 0, 1)

## Classification report of decision tree
print(classification_report(Y_test, test_pred_tree_label))

## Classification report of knn
print(classification_report(Y_test, test_pred_knn_label))

## Classification report of svm
print(classification_report(Y_test, test_pred_svm_label))

              precision    recall  f1-score   support

           0       0.85      0.90      0.88      2337
           1       0.57      0.46      0.51       663

    accuracy                           0.80      3000
   macro avg       0.71      0.68      0.69      3000
weighted avg       0.79      0.80      0.80      3000

              precision    recall  f1-score   support

           0       0.85      0.78      0.81      2337
           1       0.40      0.52      0.45       663

    accuracy                           0.72      3000
   macro avg       0.63      0.65      0.63      3000
weighted avg       0.75      0.72      0.73      3000

              precision    recall  f1-score   support

           0       0.85      0.92      0.88      2337
           1       0.59      0.42      0.49       663

    accuracy                           0.81      3000
   macro avg       0.72      0.67      0.68      3000
weighted avg       0.79      0.81      0.79      3000



## Predicting on the test with meta-learner

In [37]:
## using the meta-learner to make predictions on test
meta_learner_pred = meta_learner.predict_proba(X_test_preds)[:, 1]

## Changing likelihood to labels
meta_learner_pred_label = np.where(meta_learner_pred < 0.25, 0, 1)

## Classification report of meta-learner
print(classification_report(Y_test, meta_learner_pred_label))

              precision    recall  f1-score   support

           0       0.86      0.88      0.87      2337
           1       0.55      0.49      0.52       663

    accuracy                           0.80      3000
   macro avg       0.70      0.69      0.70      3000
weighted avg       0.79      0.80      0.79      3000



## Homogeneous Ensembles

In [1]:
import boto3
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

## Defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'data-448'
bucket = s3.Bucket(bucket_name)

## Defining the file to be read from s3 bucket
file_key = 'Chapter6/energy_data.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## Reading the csv file
energy = pd.read_csv(file_content_stream)
energy.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,1/11/16 17:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.60,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,1/11/16 17:10,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.48,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,1/11/16 17:20,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.37,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,1/11/16 17:30,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.25,733.8,92.0,6.000000,51.500000,5.0,45.410390,45.410390
4,1/11/16 17:40,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.13,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [2]:
energy.shape

(19735, 29)

## Homogeneous Ensemble: Random Samples

In [8]:
## Defining the input and target variables
X = energy.drop(columns = ['date', 'Appliances', 'lights', 'rv2'], axis = 1)
Y = energy['rv2']

## Splitting the data 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

## Standardizing the data
scaler = MinMaxScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
X_test = pd.DataFrame(scaler.fit_transform(X_test), columns = X_test.columns)

## Defining the list to store predictions
svm_preds = list()

## Defining the training dataset
train_data = pd.concat([X_train, Y_train.reset_index(drop = True)], axis = 1)

for i in range(0, 10):
    
    ## Sampling the training dataset
    train_data_loop = train_data.sample(frac = 0.8)
    X_train_loop = train_data_loop.drop(columns = ['rv2'], axis = 1)
    Y_train_loop = train_data_loop['rv2']
    
    ## Building the svm model
    svm_md = SVR(kernel = 'rbf').fit(X_train_loop, Y_train_loop)
    
    ## Storing predictions 
    svm_preds.append(svm_md.predict(X_test))
    
## Putting all the prediction in a data-frame 
svm_preds = pd.DataFrame(svm_preds).T

## Aggregation the predictions
svm_preds['Final_Pred'] = svm_preds.apply(np.mean, axis = 1)
print(svm_preds.head())

## Computing the mse of each model 
def mse_cols(col):
    return mean_squared_error(Y_test, col)

svm_preds.apply(mse_cols, axis = 0)

           0          1          2          3          4          5  \
0   9.518557   9.519556   9.522324   9.516700   9.527098   9.526861   
1  11.010240  10.868824  11.027356  11.036257  10.992611  10.887884   
2  27.471871  27.473076  27.451704  27.478305  27.446000  27.452939   
3  21.817164  21.693488  21.681324  21.612907  21.701352  21.778350   
4   8.177091   8.139316   8.112354   8.050049   8.062251   8.068672   

           6          7          8          9  Final_Pred  
0   9.577683   9.537466   9.524713   9.586076    9.535703  
1  11.054794  11.017251  10.944557  10.926070   10.976584  
2  27.479487  27.416481  27.440737  27.467043   27.457764  
3  21.690258  21.753462  21.724227  21.730906   21.718344  
4   8.104759   8.139638   8.060878   8.175563    8.109057  


0             0.716866
1             0.708245
2             0.717376
3             0.708530
4             0.710557
5             0.712813
6             0.704260
7             0.712901
8             0.704278
9             0.721713
Final_Pred    0.709359
dtype: float64

## Homogeneous Ensemble: Bagging

In [9]:
## Defining the input and target variables
X = energy.drop(columns = ['date', 'Appliances', 'lights', 'rv2'], axis = 1)
Y = energy['rv2']

## Splitting the data 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

## Standardizing the data
scaler = MinMaxScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
X_test = pd.DataFrame(scaler.fit_transform(X_test), columns = X_test.columns)

## Defining the list to store predictions
svm_preds = list()

## Defining the training dataset
train_data = pd.concat([X_train, Y_train.reset_index(drop = True)], axis = 1)

for i in range(0, 10):
    
    ## Sampling the training dataset
    train_data_loop = train_data.sample(n = train_data.shape[0], replace = True)
    X_train_loop = train_data_loop.drop(columns = ['rv2'], axis = 1)
    Y_train_loop = train_data_loop['rv2']
    
    ## Building the svm model
    svm_md = SVR(kernel = 'rbf').fit(X_train_loop, Y_train_loop)
    
    ## Storing predictions 
    svm_preds.append(svm_md.predict(X_test))
    
## Putting all the prediction in a data-frame 
svm_preds = pd.DataFrame(svm_preds).T

## Aggregation the predictions
svm_preds['Final_Pred'] = svm_preds.apply(np.mean, axis = 1)
print(svm_preds.head())

## Computing the mse of each model 
def mse_cols(col):
    return mean_squared_error(Y_test, col)

svm_preds.apply(mse_cols, axis = 0)

           0          1          2          3          4          5  \
0  21.714034  21.753279  21.802246  21.833646  21.805281  21.783345   
1  21.681860  21.637701  21.745959  21.596826  21.686089  21.751926   
2  36.559198  36.477856  36.456338  36.560859  36.501819  36.495851   
3  34.324671  34.328704  34.359113  34.279271  34.487805  34.434071   
4  20.829668  20.884484  20.881402  20.942913  20.878592  20.874555   

           6          7          8          9  Final_Pred  
0  21.738595  21.755574  21.768025  21.846339   21.780036  
1  21.736132  21.743124  21.652690  21.777686   21.700999  
2  36.632510  36.594590  36.575451  36.468495   36.532297  
3  34.235131  34.324442  34.306228  34.506638   34.358607  
4  20.835842  20.844652  20.822605  20.866285   20.866100  


0             0.539189
1             0.490504
2             0.549375
3             0.520753
4             0.499185
5             0.512478
6             0.546180
7             0.504306
8             0.524378
9             0.531421
Final_Pred    0.513199
dtype: float64